In [1]:
import os
import numpy as np
import pandas as pd

from library.Data_Preprocessing import Balance_Ratio, training_def

os.chdir('C:/Users/user/Desktop/Darui_R08621110') 
os.getcwd()

'C:\\Users\\user\\Desktop\\Darui_R08621110'

## 

### identfy all kinds of dimensional overlapping data

In [2]:
def Corner(D, label = 'GB'):
    
    Data = D.copy()
    Data['cb'] = Data[Data.columns[1:-1]].apply(lambda x: '_'.join(x.astype(str)), axis = 1)
    Data[f'{label}_mean'] = Data.groupby('cb')[label].transform('mean')
    Data[f'{label}_count'] = Data.groupby('cb')[label].transform('count')
    Corner_overlap = Data.copy() 
    
    return Corner_overlap


def Kind(Corner_overlap, label = 'GB'):

    Data = Corner_overlap
    
    Kinds_0 = Data[Data[f'{label}_mean'] == 0].sort_values(f'{label}_count') # represent all data in the same cb are good  
    Kinds_1 = Data[Data[label] == 1].sort_values(f'{label}_count') # represent bad data
    Kinds = pd.concat([Kinds_1, Kinds_0]) # reprecent all types of 'cb'
    Kinds_after_duplicate = Kinds.drop_duplicates(subset = ['cb']).reset_index(drop = True) 
    # each 'cb' only keep one data(the first)
    
    return Kinds_after_duplicate


def Dictionary_Build(Data):
    
    Corner_overlap = Corner(Data)
    Kinds = Kind(Corner_overlap).copy()

    ## reorder cols
    cols = Kinds.columns.tolist()
    cols = cols[0:1] + cols[-4:] 
    Dictionary = Kinds[cols] #select id, GB, cb, GB_mean, GB_count
    Dictionary['G_count'] = 0 # add new column 
    Dictionary=Dictionary.reset_index(drop=True)
    for i in range(len(Dictionary)):
        Dictionary['G_count'][i] = Dictionary.GB_count[i] - Dictionary.GB[i] # represent??
    
    return Dictionary

### relabel overlapping data

In [3]:
def Remove_SD(Data, count = 1, label = 'GB'):
    
    Corner_Overlap = Data
    RSD = pd.concat([Data[Data[label] == 1], Data[(Data[label] == 0) & (Data[f'{label}_count'] > count)]]) 
    # remove good data that only show once
    return RSD


def Corner_Blend(Data, ratio = 0.002, label = 'GB'):
    
    D_1 = Data[(Data[f'{label}_mean'] <= 1) & (Data[f'{label}_mean'] >= ratio)].sort_values(f'{label}_count') # not relabel case
    D_1[[label]] = 1  #if G_mean > ratio ==> relabel all data as bad
    D_2 = Data[(Data[f'{label}_mean'] < ratio)].sort_values(f'{label}_count')
    D_2[[label]] = 0  # otherwise relabel all data as good
    Training_new = pd.concat([D_1,D_2]).iloc[:,:-3]
    
    return Training_new

## 

### loading training data

In [5]:
##### training data of each month #####
training_month = range(1, 7)
train_runhist = {}
for i in training_month:
    train_runhist[f'm{i}'] = pd.read_csv(f'train_runhist_m{i}.csv').iloc[:, 1:]
    print(f'Dimension of month {i}:', train_runhist[f'm{i}'].shape)
train_runhist['all'] = training_def(train_runhist, training_month)

##### training & testing data #####
test_runhist = pd.read_csv('test_runhist.csv').iloc[:, 1:]
print('\nDimension of training data:', train_runhist['all'].shape,
      '\nDimension of testing data:', test_runhist.shape)

Dimension of month 1: (17735, 85)
Dimension of month 2: (39009, 91)
Dimension of month 3: (60396, 93)
Dimension of month 4: (57743, 100)
Dimension of month 5: (48649, 94)
Dimension of month 6: (7792, 72)

Dimension of training data: (231324, 135) 
Dimension of testing data: (43243, 135)


### relabel the training data by month

In [8]:
##### training data of each month #####
new_runhist = {}
for i in training_month:
    overlap = Corner(train_runhist[f'm{i}'])
    new_runhist[f'm{i}'] = Corner_Blend(overlap, 1/100000)
    new_runhist[f'm{i}'].to_csv(f'relabel_runhist_m{i}.csv')
    print(f'Month {i}:')
    print('balance ratio:', Balance_Ratio(new_runhist[f'm{i}']), 
          '# original bad:', sum(train_runhist[f'm{i}'].GB), '# total bad:', sum(new_runhist[f'm{i}'].GB))
    
##### the whole training data #####
new_runhist['all'] = training_def(new_runhist, training_month)
print('\nAll training data after overlap relabel:', new_runhist['all'].shape, 
      ', balance ratio:', Balance_Ratio(new_runhist['all']), ', # total bad:', sum(new_runhist['all'].GB))

Month 1:
balance ratio: 1181.0 # original bad: 7 # total bad: 15
Month 2:
balance ratio: 533.0 # original bad: 31 # total bad: 73
Month 3:
balance ratio: 443.0 # original bad: 46 # total bad: 136
Month 4:
balance ratio: 433.0 # original bad: 54 # total bad: 133
Month 5:
balance ratio: 415.0 # original bad: 51 # total bad: 117
Month 6:
balance ratio: 1112.0 # original bad: 7 # total bad: 7

All training data after overlap relabel: (231324, 135) , balance ratio: 480.0 , # total bad: 481


### cauculate the number of feature combination types 

In [12]:
##### training data of each month #####
kinds = {}
for i in training_month:
    overlap = Corner(train_runhist[f'm{i}'])
    kinds[f'm{i}'] = Kind(overlap)
    kinds[f'm{i}'].to_csv(f'kind_m{i}.csv')
    print(f'Month {i}:')
    print('# kinds:', len(kinds[f'm{i}']), ', # bad kinds:', len(kinds[f'm{i}'][kinds[f'm{i}'].GB == 1]))

##### the whole training data & testing data #####
train_overlap = Corner(train_runhist['all'])
kinds['all'] = Kind(train_overlap)
test_overlap = Corner(test_runhist)
test_kinds = Kind(test_overlap)
print('\n# kinds in all training data:', len(kinds['all']), ', # bad kinds:', len(kinds['all'][kinds['all'].GB == 1]))
print('\n# kinds in testing data:', len(test_kinds), ', # bad kinds:', len(test_kinds[test_kinds.GB == 1]))

Month 1:
# kinds: 11355 , # bad kinds: 7
Month 2:
# kinds: 23399 , # bad kinds: 31
Month 3:
# kinds: 32264 , # bad kinds: 46
Month 4:
# kinds: 31732 , # bad kinds: 54
Month 5:
# kinds: 28517 , # bad kinds: 51
Month 6:
# kinds: 7121 , # bad kinds: 7

# kinds in all training data: 131705 , # bad kinds: 196

# kinds in testing data: 26524 , # bad kinds: 37
